## Accenture - Supply Chain


In [28]:
# Packaging management
import numpy as np
import pandas as pd
import csv 
from sklearn.linear_model import LinearRegression as lm
import statsmodels

# Data Preparation

In [25]:
## Distance matrix preparation

## Read raw data from csv files
pd_cities_raw = pd.read_csv("datas/cities_data.csv" ,sep=";")

city_list_s = pd_cities_raw['city_from_name'].unique()
city_list_d = pd_cities_raw['city_to_name'].unique()
all_cities= list(set(city_list_s) | set(city_list_d))

city_dict = {all_cities[i]:i for i in range(0,len(all_cities))}
n_cities = len(all_cities)

distance_mat = [[0]*n_cities]*n_cities


for index, row in pd_cities_raw.iterrows():
    s = city_dict[row["city_from_name"]]
    d = city_dict[row["city_to_name"]]
    distance = row["distance"]
    distance_mat[s][d] = distance

In [26]:
## Product Matrix

pd_products_raw = pd.read_csv("datas/product_attributes.csv", sep=",")

##dataframe quering example  !(return a string value)
# pd_products_raw.loc[pd_products_raw["product_id"]==1672707]["material_handling"].to_string(index=False)


In [21]:
#Needs Revision
customs_procedure =pd_cities_raw['customs_procedures']
print(customs_procedure)



KeyError: 'customs_procedures'

In [14]:
## Probability dictionary calculation
pd_orders_raw=pd.read_csv("datas/orders.csv",sep=";")

pd_orders_raw.loc[pd_orders_raw["origin_port"] == "BCN", "origin_port"] = "Barcelona"
pd_orders_raw.loc[pd_orders_raw["origin_port"] == "ATHENAS", "origin_port"] = "Athens"


cities_origin = pd_orders_raw['origin_port'].unique()
cities_hub = pd_orders_raw['logistic_hub'].unique()
cities_customer =  pd_orders_raw['customer'].unique()
pd_origin_count = pd_orders_raw.groupby(['origin_port', 'late_order']).size()
pd_hub_count = pd_orders_raw.groupby(['logistic_hub', 'late_order']).size()
pd_customer_count = pd_orders_raw.groupby(['customer', 'late_order']).size()

city_count_origin =  pd_orders_raw.groupby(['origin_port']).size()
city_count_hub = pd_orders_raw.groupby(['logistic_hub']).size()
city_count_customer = pd_orders_raw.groupby(['customer']).size()

delay_rate_origin = {all_cities[i]:0 for i in range(0,len(all_cities))}
delay_rate_hub = {all_cities[i]:0 for i in range(0,len(all_cities))}
delay_rate_customer = {all_cities[i]:0 for i in range(0,len(all_cities))}


origin_count_dict =  pd_orders_raw.groupby(['origin_port','late_order']).size().to_dict()
hub_count_dict =  pd_orders_raw.groupby(['logistic_hub','late_order']).size().to_dict()
customer_count_dict =  pd_orders_raw.groupby(['customer','late_order']).size().to_dict()

In [23]:
for key, values in origin_count_dict:
    total = city_count_origin[key]
    if (values == True):
        true_count_origin = origin_count_dict[key,values]
        delay_rate_origin[key] = true_count_origin/total
print("origin city")
print(delay_rate_origin)
for key, values in hub_count_dict:
    total = city_count_hub[key]
    if (values == True):
        true_count_origin = hub_count_dict[key,values]
        delay_rate_hub[key] = true_count_origin/total
print("hub city")
print(delay_rate_hub)
for key, values in customer_count_dict:
    total = city_count_customer[key]
    if (values == True):
        true_count_origin = customer_count_dict[key,values]
        delay_rate_customer[key] = true_count_origin/total
print("destination city")
print(delay_rate_customer)

origin city
{'Barcelona': 0.2702263870848024, 'Turin': 0.19692020531964535, 'Vienna': 0.20310734463276836, 'Zaragoza': 0.3497337053005326, 'Berlin': 0.19584479071188513, 'Cologne': 0.1831454918032787, 'Marseille': 0.32769390942217597, 'Bremen': 0.1761786600496278, 'Helsinki': 0.3217884130982368, 'Rotterdam': 0.16921047383841328, 'Amsterdam': 0.19564012562349897, 'Lyon': 0.3249023236685174, 'Copenhagen': 0.2667623539060898, 'Rome': 0.22538860103626943, 'Porto': 0.31744604316546765, 'Dusseldorf': 0.18381530595941845, 'Paris': 0.30714501967907964, 'Prague': 0.20864197530864198, 'Hanover': 0.18755555555555556, 'Bratislava': 0.3308839902026275, 'Hamburg': 0.20204841713221602, 'Munich': 0.1941458574499752, 'Budapest': 0.2088235294117647, 'Bordeaux': 0.3195558297347316, 'Warsaw': 0.22755173068404733, 'Athens': 0.3741508954193759, 'Venlo': 0.18738396624472572, 'Stockholm': 0.28258221680876977, 'Valencia': 0.23809523809523808, 'Lille': 0.3595823575331772, 'Milan': 0.20382626680455015, 'Madrid':

In [32]:
threepl_list = pd_orders_raw['3pl'].unique()
procedures = pd_orders_raw['customs_procedures'].unique()

n3pl = len(threepl_list)
nprocedures= len(procedures)

dict_3pl = {threepl_list[i]:i for i in range(0,len(threepl_list))}
dict_procedures = {procedures[i]:i for i in range(0,len(procedures))}

pd_orders_raw['3pl'] = pd_orders_raw['3pl'].factorize()[0]
pd_orders_raw['customs_procedures'] = pd_orders_raw['customs_procedures'].factorize()[0]



3

# Data Test

In [ ]:
# Correlation map

# 1: convert dictionaries into dataframe

jet= plt.get_cmap('jet')
colors = iter(jet(np.linspace(0,1,10)))

def correlation(df, variables, n_rows, n_cols):
    fig = plt.figure(figsize=(15,10))
    for i, var in enumerate(variables):
        ax = fig.add_subplot(n_rows,n_cols,i+1)
        asset = df.loc[:,var]
        sns.scatterplot(df["price"], asset, c = next(colors))
        ax.set_xlabel("price")
        ax.set_ylabel("{}".format(var))
        ax.set_title(var +" vs price")
        plt.xticks(rotation=30)
    fig.tight_layout()
    plt.show()

# Data Training